# Verify Ensemble Submission - No Overlaps

Verify the current submission has no overlapping trees before submitting.

In [1]:
import numpy as np
import pandas as pd
from shapely.geometry import Polygon
from shapely.strtree import STRtree
from shapely import affinity
from decimal import Decimal
import math

print('Libraries loaded')

Libraries loaded


In [2]:
# Tree polygon template
TX = np.array([0, 0.125, 0.0625, 0.2, 0.1, 0.35, 0.075, 0.075, -0.075, -0.075, -0.35, -0.1, -0.2, -0.0625, -0.125])
TY = np.array([0.8, 0.5, 0.5, 0.25, 0.25, 0, 0, -0.2, -0.2, 0, 0, 0.25, 0.25, 0.5, 0.5])

def get_polygon(x, y, deg):
    """Get polygon for a tree at position (x, y) with rotation deg."""
    rad = np.radians(deg)
    c, s = np.cos(rad), np.sin(rad)
    px = TX * c - TY * s + x
    py = TX * s + TY * c + y
    return Polygon(zip(px, py))

def strip(val):
    """Remove 's' prefix from value."""
    return float(str(val).replace('s', ''))

def has_overlap(trees_data):
    """Check if any trees in the configuration overlap."""
    if len(trees_data) <= 1:
        return False
    
    polys = []
    for _, row in trees_data.iterrows():
        x = strip(row['x'])
        y = strip(row['y'])
        deg = strip(row['deg'])
        polys.append(get_polygon(x, y, deg))
    
    tree_index = STRtree(polys)
    for i, poly in enumerate(polys):
        indices = tree_index.query(poly)
        for idx in indices:
            if idx == i:
                continue
            if poly.intersects(polys[idx]) and not poly.touches(polys[idx]):
                return True
    return False

print('Overlap detection functions defined')

Overlap detection functions defined


In [3]:
# Load submission
df = pd.read_csv('/home/submission/submission.csv')
print(f'Loaded {len(df)} rows')

# Check for overlaps in each N
overlap_found = []
for n in range(1, 201):
    group = df[df['id'].str.startswith(f'{n:03d}_')]
    if has_overlap(group):
        overlap_found.append(n)
        print(f'OVERLAP in N={n}')

if overlap_found:
    print(f'\n*** OVERLAPS FOUND in N: {overlap_found} ***')
else:
    print('\n*** NO OVERLAPS - SUBMISSION IS VALID ***')

Loaded 20100 rows
OVERLAP in N=12
OVERLAP in N=16
OVERLAP in N=20


OVERLAP in N=35
OVERLAP in N=36


OVERLAP in N=50
OVERLAP in N=52


OVERLAP in N=67


OVERLAP in N=77
OVERLAP in N=78


...

OVERLAP in N=128
OVERLAP in N=137


OVERLAP in N=157
OVERLAP in N=160
OVERLAP in N=164
OVERLAP in N=166


OVERLAP in N=173


OVERLAP in N=187



*** OVERLAPS FOUND in N: [12, 16, 20, 35, 36, 50, 52, 67, 77, 78, 92, 95, 103, 116, 117, 118, 124, 128, 137, 157, 160, 164, 166, 173, 187] ***


In [4]:
# Calculate total score
from numba import njit

@njit
def make_polygon_template():
    tw=0.15; th=0.2; bw=0.7; mw=0.4; ow=0.25
    tip=0.8; t1=0.5; t2=0.25; base=0.0; tbot=-th
    x=np.array([0,ow/2,ow/4,mw/2,mw/4,bw/2,tw/2,tw/2,-tw/2,-tw/2,-bw/2,-mw/4,-mw/2,-ow/4,-ow/2],np.float64)
    y=np.array([tip,t1,t1,t2,t2,base,base,tbot,tbot,base,base,t2,t2,t1,t1],np.float64)
    return x,y

@njit
def score_group(xs, ys, degs, tx, ty):
    n = xs.size
    V = tx.size
    mnx = 1e300; mny = 1e300; mxx = -1e300; mxy = -1e300
    for i in range(n):
        r = degs[i] * math.pi / 180.0
        c = math.cos(r); s = math.sin(r)
        xi = xs[i]; yi = ys[i]
        for j in range(V):
            X = c * tx[j] - s * ty[j] + xi
            Y = s * tx[j] + c * ty[j] + yi
            if X < mnx: mnx = X
            if X > mxx: mxx = X
            if Y < mny: mny = Y
            if Y > mxy: mxy = Y
    side = max(mxx - mnx, mxy - mny)
    return side * side / n

tx, ty = make_polygon_template()

total_score = 0.0
for n in range(1, 201):
    group = df[df['id'].str.startswith(f'{n:03d}_')]
    xs = np.array([strip(x) for x in group['x']], dtype=np.float64)
    ys = np.array([strip(y) for y in group['y']], dtype=np.float64)
    degs = np.array([strip(d) for d in group['deg']], dtype=np.float64)
    total_score += score_group(xs, ys, degs, tx, ty)

print(f'\nTotal Score: {total_score:.6f}')
print(f'Target: 68.922808')
print(f'Gap: {total_score - 68.922808:.6f}')


Total Score: 70.676092
Target: 68.922808
Gap: 1.753284


In [ ]:
# Fix the overlaps by replacing with valid configurations from sources\nimport os\n\n# List of sources\nsources = [\n    '/home/code/preoptimized_submission.csv',\n    '/home/code/datasets/santa-2025.csv',\n    '/home/code/datasets/71.97.csv',\n    '/home/code/datasets/72.49.csv',\n    '/home/code/datasets/submission.csv',\n    '/home/code/datasets/jazivxt_output/submission.csv',\n    '/home/code/datasets/eazy_output/submission.csv',\n    '/home/code/datasets/ashraful_output/submission.csv',\n]\n\n# Load all sources\nall_sources = {}\nfor src in sources:\n    if os.path.exists(src):\n        all_sources[src] = pd.read_csv(src)\n        print(f'Loaded {src}')\n\nprint(f'\\nTotal sources: {len(all_sources)}')"}

In [ ]:
# For each N, find the best valid (no overlap) configuration\nbest = {n: {'score': 1e300, 'data': None, 'src': None} for n in range(1, 201)}\n\nfor src_path, src_df in all_sources.items():\n    src_name = os.path.basename(src_path)\n    src_df = src_df.copy()\n    src_df['N'] = src_df['id'].astype(str).str.split('_').str[0].astype(int)\n    \n    for n, group in src_df.groupby('N'):\n        if n < 1 or n > 200:\n            continue\n        \n        # Check for overlaps\n        group_clean = group.drop(columns=['N'])\n        if has_overlap(group_clean):\n            continue  # Skip configurations with overlaps\n        \n        # Calculate score\n        xs = np.array([strip(x) for x in group['x']], dtype=np.float64)\n        ys = np.array([strip(y) for y in group['y']], dtype=np.float64)\n        degs = np.array([strip(d) for d in group['deg']], dtype=np.float64)\n        sc = score_group(xs, ys, degs, tx, ty)\n        \n        if sc < best[n]['score']:\n            best[n]['score'] = float(sc)\n            best[n]['data'] = group_clean.copy()\n            best[n]['src'] = src_name\n\nprint('Processed all sources with overlap checking')"}

In [ ]:
# Check which N values have valid configurations\nmissing = []\nfor n in range(1, 201):\n    if best[n]['data'] is None:\n        missing.append(n)\n        print(f'WARNING: No valid configuration for N={n}')\n\nif missing:\n    print(f'\\nMissing N values: {missing}')\nelse:\n    print('All N values have valid configurations')"}